<!-- dom:TITLE: Fundamental cardiac mechanics -->
# Fundamental cardiac mechanics
<!-- dom:AUTHOR: Joakim Sundnes -->
<!-- Author: -->  
**Joakim Sundnes**

Date: **June 2024**

* Introduce the foundation of non-linear solid mechanics

  * Fundamental principles and methods

  * Introduce the most important quantities


* Show how the equations are adapted to heart tissue mechanics

* Solve the tissue mechanics equations using Fenics

## Goals for this intro lecture
Dependent on background:
  1. No background in continuum mechanics 
     * Get an understanding of the physical basis of the elasticity equation and other classical models
  2. Familiar with continuum mechanics, no background in non-linear solid mechanics:
      * Explain the key challenges and differences from small-deformation elasticity, introduce widely used stress and strain tensors, explain fundamental laws
  3. Familiar with non-linear solid mechanics: 
      * Introduce material laws used for heart tissue

## Suggested further reading
* General biomechanics:
  Y.C. Fung. Biomechanics: Mechanical Properties of Living Tissues

* Non-linear solid mechanics:
  G. A. Holzapfel: Nonlinear Solid Mechanics: A Continuum Approach for Engineering

## Outline
* From cell to tissue
  * The continuum approach
* Fundamentals of mechanics
  * Stress
  * Strain
  * Fundamental equations
* Modeling soft tissues
  * Non-linear solid mechanics
  * Material laws for the heart

## From cell to tissue (1)
We have seen how we can model the contraction of a single myocyte...


<!-- dom:FIGURE:[./figs/myocyte.png,width=300 frac=0.4] -->
<!-- begin figure -->

<p></p>
<img src="./figs/myocyte.png" width=300>

<!-- end figure -->


<!-- dom:FIGURE:[./figs/F_SL_transients.png,width=500 frac=0.8] -->
<!-- begin figure -->

<p></p>
<img src="./figs/F_SL_transients.png" width=500>

<!-- end figure -->

## From cell to tissue (2)
... but the
heart contains  around 10 billion myocytes, embedded in a complex compound of
proteins, fluids, other cells, etc.


<!-- dom:FIGURE:[./figs/tissue_combined.png,width=500 frac=0.9] -->
<!-- begin figure -->

<p></p>
<img src="./figs/tissue_combined.png" width=500>

<!-- end figure -->

## The continuum approach
 * The tissue is viewed as continuous and homogeneous, neglecting the discrete, cell-based nature

 * Takes a macro-scopic, top-down view of the tissue

 * The same approach is used for modeling other materials, neglecting the molecular/atomic structure

 * We can apply the standard tools of continuum mechanics, which have been developed over centuries

## Some additional challenges in biomechanics
 * Large deformations

 * Strongly non-linear material behavior

 * Active properties

 * Growth and remodeling

 * (The cell scale is much larger than the atomic scale, which means the continuum approach may not always be justified)

## The three generic steps of continuum mechanics

1. Formulate the balance equations
  * Mass
  * Momentum
  * Energy

2. Incorporate material behavior
  * Constitutive laws
  * Relevant simplifications

3. Solve the equations

## Comments on Step 3
  * Classic approach with pen and paper:
    * Make the right simplifying assumptions, lots of algebra
  * Modern computational approach: 
    * Solve the "full" PDE model (requires a precise problem description)

## Solid mechanics (1)

Our focus is on solid mechanics, which typically involves the following:
 * Apply some load (force) to a structure:

   * What are the internal forces (stresses)?

   * Will the structure hold or break?

   * How much does the structure deform?

## Solid mechanics (2)

Heart mechanics is similar, but with some differences:
 * Load is dynamic (blood pressure)

 * Material properties are dynamic (cells contract)

 * Material parameters are **very** uncertain

 * What is the deformation (volume change) for a given load

 * What are the internal forces (related to growth and remodeling)

## Solid mechanics (3)

* The key variables in solid mechanics problems are stresses and strains

* Stress = force per area, strain = relative deformation

* Important distinction between small and large deformations:

* Small deformation; neglect the change in area as the material deforms

* Large deformations (> 5% strain); need to consider change in area

* Small deformation problems are often linear, large deformations give nonlinear problems

## Solid mechanics (4)

In short, the field of solid mechanics has three main parts:
* Kinematics; the description of motion and deformation of the material (i.e. strains)

* Balance laws; fundamental equations based on balance of mass and momentum, or equilibrium of forces/stresses in the static case

* Constitutive laws; experimentally derived laws that relate stresses to strains.

## Kinematics

How do we quantify deformation/change of shape?
* What is strain, really?

* Important measures of strain and deformation in non-linear solid mechanics:

  * The deformation gradient ($\boldsymbol{F}$)

  * The right Cauchy-Green tensor ($\boldsymbol{C}$)

  * The Green-Lagrange strain tensor ($\boldsymbol{E}$)

## Deformation and displacement (1)

<!-- dom:FIGURE:[./figs/Lagrangian.png,width=300 frac=0.4] -->
<!-- begin figure -->

<p></p>
<img src="./figs/Lagrangian.png" width=300>

<!-- end figure -->


* The path of each particle: $\boldsymbol{x}(\boldsymbol{X},t)$

* Initial position: $\boldsymbol{x}(\boldsymbol{X}, 0) = \boldsymbol{X}$ (particle label)

* Displacement field $\boldsymbol{u}$:

$$
\boldsymbol{u}(\boldsymbol{X},t) = \boldsymbol{x}(\boldsymbol{X},t)-\boldsymbol{X}
$$

## Deformation and displacement (2)

* The displacement $\boldsymbol{u}$ is often the primary unknown in solid mechanics

* $\boldsymbol{u}$ contains everything we need to know about the deformation;

  * Change of shape ("true" deformation)

  * Rigid body motion (rotation and translation)


* Our goal is to relate internal forces (stresses) to deformations

* Rigid body motion does not give rise to internal forces

* We need a measure of deformation that only contains *change of shape*

## The deformation gradient

We are interested in relative displacement between different points.
It makes sense to take the derivative of the mapping:
$$
\boldsymbol{F} = \frac{\partial{\boldsymbol{x}}}{{\partial X}} = \nabla\boldsymbol{x} = I + \nabla\boldsymbol{u}
$$
This is the *deformation gradient*, a fundamental quantity in nonlinear
solid mechanics. In particular, an infinitesimal line segment $\boldsymbol{dX}$ in
the reference configuration deforms according to
$$
\boldsymbol{dx} = \boldsymbol{F}\boldsymbol{dX}
$$

## The right Cauchy-Green tensor
* The deformation gradient includes both rotation and change of shape.

* Rotation does not induce internal forces, and must be removed to get a measure of of pure change of shape.

* According to the Polar decomposition theorem, if $\boldsymbol{F}$ is non-singular we have
  $\boldsymbol{F} = \boldsymbol{RU}$, where $\boldsymbol{R}$ is an orthogonal rotation tensor and $\boldsymbol{U}$
  is symmetric and contains no rotation.

* We introduce $\boldsymbol{C} = \boldsymbol{F}^T\boldsymbol{F} = (\boldsymbol{RU})^T(\boldsymbol{RU}) = \boldsymbol{U}^T \boldsymbol{R}^T\boldsymbol{RU} = \boldsymbol{U}^T\boldsymbol{U}$

* $\boldsymbol{C}$ is called the *right Cauchy-Green tensor*

## The Green-Lagrange strain tensor (1)
* We are now ready to define strain

* Look at a small line segment $\boldsymbol{dX}$, which deforms to $\boldsymbol{dx}$

* If $\|\boldsymbol{dx}\| = \|\boldsymbol{dX}\|$: pure rotation

* Suitable strain measure arises from

$$
\begin{align*}
\|\boldsymbol{dx}\|^2 - \|\boldsymbol{dX}\|^2 &= \boldsymbol{dx}^T\boldsymbol{dx} - \boldsymbol{dX}^T\boldsymbol{dX} \\
&= (\boldsymbol{F}\boldsymbol{dX})^T\boldsymbol{F}\boldsymbol{dX} - \boldsymbol{dX}^T\boldsymbol{dX} \\
& = \boldsymbol{dX}^T\boldsymbol{F}^T\boldsymbol{F}\boldsymbol{dX} - \boldsymbol{dX}^T\boldsymbol{dX} \\
& = \boldsymbol{dX}^T(\boldsymbol{C}-\boldsymbol{I})\boldsymbol{dX} = 2\boldsymbol{dX}^T\boldsymbol{E}\boldsymbol{dX}
\end{align*}
$$

$\boldsymbol{E}$ is the important quantity holding strain information.

## The Green-Lagrange strain tensor (2)
The Green Lagrange strain tensor is the most commonly used
strain tensor for large elastic deformations. We have
$$
\boldsymbol{E} = \frac{1}{2}(\boldsymbol{C}-\boldsymbol{I}) = \frac{1}{2}(\boldsymbol{F}^T\boldsymbol{F}-\boldsymbol{I})
$$
or, in index notation:
$$
E_{ij} = \frac{1}{2}(\frac{\partial x_k}{\partial X_i}\frac{\partial x_k}{\partial X_j}-\delta_{ij})
$$
Notice that we use the summation convention, so when index $k$ is repeated in
the same term, we set $k=1,2,3$ and sum over it.

## Relation to small deformation strain
We can expand the expression for $E_{ij}$:

$$
\begin{align*}
E_{ij} = \frac{1}{2}(\frac{\partial x_k}{\partial X_i}\frac{\partial x_k}{\partial X_j}-\delta_{ij})
&= \frac{1}{2}(\frac{\partial u_i}{\partial X_j} +
\frac{\partial u_j}{\partial X_i} +
\frac{\partial u_k}{\partial X_i}\frac{\partial u_k}{\partial X_j})
\end{align*}
$$

The derivation is left as an exercise. Use the definition $\boldsymbol{u}=\boldsymbol{x}-\boldsymbol{X}$.

If deformations are small we can neglect higher order terms, and
we have $\boldsymbol{x} \approx \boldsymbol{X}$. This gives
$$
E_{ij} \approx \frac{1}{2}(\frac{\partial u_i}{\partial X_j} +
\frac{\partial u_j}{\partial X_i})
\approx \frac{1}{2}(\frac{\partial u_i}{\partial x_j} +
\frac{\partial u_j}{\partial x_i})
$$
which is recognized as the standard linear strain tensor $\varepsilon_{ij}$
used for small strains.

## Fundamental laws in continuum mechanics

* Conservation of mass
* Conservation of momentum
* Conservation of energy


## Conservation of momentum (1)

* In solid mechanics, the key balance equation is conservation of momentum

  * Mass conservation implicitly satisfied

  * Energy balance usually not considered (although highly relevant in
     biomechanics)

## Conservation of momentum (2)
Newton's second law applied to a continuous and deforming material.

$$
\sum F = ma (\approx 0)
$$

* This force balance principle applies to *every* part of the material

  * Consider an arbitrary piece of the continuum body, sum up the forces
     to arrive at the balance law

  * Two types of forces; **body forces** and **surface forces**

## Body forces
<!-- FIGURE:[figs/body_forces,width=400 frac=0.3] -->

* Body forces $\boldsymbol{b}$ are "distant" forces acting in each point of the body

* Example: gravity $\boldsymbol{b} = \boldsymbol{g}$

* Example: centrifugal force $\boldsymbol{b} = \boldsymbol{\omega}\times\boldsymbol{\omega}\times\boldsymbol{r}$

* Total force:

$$
\boldsymbol{B} = \int\limits_{\hbox{body}}\varrho \boldsymbol{b} dV
$$

## Surface forces (1)
<!-- dom:FIGURE:[figs/surface_forces.png,width=400 frac=0.3] -->
<!-- begin figure -->

<p></p>
<img src="figs/surface_forces.png" width=400>

<!-- end figure -->

## Surface forces (2)

* Distributed along the surface of a body or of an internal part of a body

* Stress = force per unit area, $\boldsymbol{s}(\boldsymbol{x} ,t)$

* Total force:

$$
\boldsymbol{S}(\boldsymbol{x},t) = \int\limits_{\hbox{surface}} \boldsymbol{s}(\boldsymbol{x} ,t) dA
$$

## The stress vector

  * Stress is force per unit area (vector)

  * The stress vector depends on the orientation of the area

  * That is, the stress at a point on a surface depends on the location of the point (on the surface) and on the orientation of the surface at that point

## Stress in a rod
<!-- dom:FIGURE:[figs/rodstress.png,width=600 frac=0.8] -->
<!-- begin figure -->

<p></p>
<img src="figs/rodstress.png" width=600>

<!-- end figure -->

## Observations

  * The stress at the bullet point was in one case $F/A$ ($A$: area) and in another case 0!

  * "Stress" means stress at a *point* on a  *surface*

  * The surface orientation (normal vector $\boldsymbol{n}$) is needed for stress vector computations

## Stress vector computation

  * The stress vector depends on space, time and the orientation  (unit outward normal vector $\boldsymbol{n}$) of     the surface on which the stress vector acts

  * Notation: $\boldsymbol{s} (\boldsymbol{r},t;\boldsymbol{n})$ ($\boldsymbol{s}$ varies in $\mathbb{R}^7$,       i.e., with 7 parameters!)

  * Cauchy's 1. law (Cauchy's stress theorem):

$$
\boldsymbol{s} (\boldsymbol{r},t;\boldsymbol{n}) = \boldsymbol{n}(\boldsymbol{r},t)\cdot
\boldsymbol{\sigma}(\boldsymbol{r},t)
$$

($\Rightarrow$ $\boldsymbol{s}$ has a simple (linear) dependence on $\boldsymbol{n}$)

## The stress tensor

  * The quantity $\boldsymbol{\sigma}$ or $\sigma_{ij}$ in Cauchy's 1. law is called the stress tensor

  * $\boldsymbol{\sigma}$ contains 9 entries:

$$
\boldsymbol{\sigma} = \left(\begin{array}{ccc} \sigma_{xx} & \sigma_{xy} & \sigma_{xz}\\ \sigma_{yx} & \sigma_{yy} & \sigma_{yz}\\ \sigma_{zx} & \sigma_{zy} & \sigma_{zz}
\end{array}\right)
$$

The entries have a physical interpretation, but represent mainly
ingredients in a tool (Cauchy's 1. law) for computing the stress
vector at an arbitrary surface

## Stress tensor in a rod (1)
<!-- dom:FIGURE:[figs/rod.png,width=600 frac=0.7] -->
<!-- begin figure -->

<p></p>
<img src="figs/rod.png" width=600>

<!-- end figure -->


<!-- \centerline{\includegraphics[width=0.6\linewidth]{figs/rod}} -->


  * Uni-axial tension force

  * How can we find the stress tensor in this case?

  * General approach: solve the governing PDE with boundary conditions (possible even analytically!)

  * Or: use physical reasoning to guess at a stress tensor (usually difficult, but possible in this case)

## Stress tensor in a rod (2)

  * Cutting the body along coordinate planes ($x=\hbox{const}$, $y=\hbox{const}$, $z=\hbox{const}$)
<!-- \centerline{\includegraphics[width=0.4\linewidth]{figs/rodstress}} suggests -->

$$
\boldsymbol{\sigma} = \left(\begin{array}{ccc} F/A & 0 & 0\\ 0 & 0 & 0\\ 0 & 0 & 0
\end{array}\right)
$$

## Stress tensor in a rod (3)

  * How can we know that this guess is correct?

  * Rhysical reasoning indicates such a stress tensor, but only the solution of a full model for elastic deformation can tell if our assumption of $\boldsymbol{\sigma}$ is correct

  * Classically, such guesses based on physical reasoning were  required to treat the problem analytically

  * Today, such guesses are crucial to assess whether numerical results are reasonable

## Deriving the balance law - force equilibrium
For an arbitrary volume $V$ inside a material, Newton’s 2nd law reads


$$
\frac{d}{dt}\int_V \rho \boldsymbol{v} dV = \sum \boldsymbol{f}
$$


where $\rho$ is density, $\boldsymbol{v}$ velocity, and $\sum f$ is the
total force: surface forces + body forces.

## Equilibrium of forces
In solid mechanics, it is common to neglect inertia effects:


$$
\rho \boldsymbol{v} \approx 0
$$

which reduces Newton's second law to a force equilibrium:
$$
\sum \boldsymbol{f} = 0
$$

## Two types of forces
As noted earlier, two types of forces act on the material volume $V$:

* Surface forces:
  $$
  \int\limits_{\partial V} \boldsymbol{\sigma}\cdot\boldsymbol{n}dS
  $$

* Body forces (e.g. gravity):
  $$
  \int\limits_{V} \varrho\boldsymbol{b}dV
  $$

## Sum up the forces
We have

$$
\int\limits_{\partial V} \boldsymbol{\sigma}\cdot\boldsymbol{n}dS + \int\limits_{V} \varrho\boldsymbol{b}dV = 0
$$

Applying Gauss' theorem to the first term gives

$$
\int\limits_{V} \nabla\cdot\boldsymbol{\sigma} dV + \int\limits_{V} \varrho\boldsymbol{b}dV = 0
$$

## Cauchy's equation of motion
Since the volume is arbitrary we must have

$$
\nabla\cdot\boldsymbol{\sigma} + \varrho\boldsymbol{b} = 0 .
$$

* This is the static (equilibrium) version of Cauchy's equation of motion.

* In heart mechanics the effects of gravity are negligible, and we are left with

$$
\nabla\cdot\boldsymbol{\sigma} = 0
$$

## Comments on Cauchy's equation (1)
* The equilibrium equation derived above is completely valid for all
   materials in equilibrium

* The same derivation is used in fluid mechanics, but in this
  case the inertia must usually be considered (Navier-Stokes)

* Inconvenient to use for large deformations, since Cauchy stress $\boldsymbol{\sigma}$ is defined relative to the deformed area, which is unknown

* Differentiation (the divergence operator) is also performed relative to the deformed coordinates of the material, which are not known

## Comments on Cauchy's equation (2)
For large-deformation solid mechanics, a so-called Lagrangian approach is common:
* Map all quantities and operations back to the undeformed geometry of the material

* Introduce alternative stress tensors that are computed on the undeformed geometry

## Comments on Cauchy's equation (3)
The divergence of a tensor is a vector:

$$
\begin{align*}
\nabla\cdot \boldsymbol{\sigma} &= \nabla\cdot
\left(\begin{array}{ccc} \sigma_{xx}
& \sigma_{xy} & \sigma_{xz}\\ \sigma_{yx} & \sigma_{yy} &
\sigma_{yz}\\ \sigma_{zx} & \sigma_{zy} & \sigma_{zz} \end{array}
\right) 
=
\left(\begin{array}{c} \frac{\partial \sigma_{xx}}{\partial x}
+ \frac{\partial \sigma_{xy}}{\partial y} +\frac{\partial
\sigma_{xz}}{\partial z}\\ \frac{\partial \sigma_{yx}}{\partial x}
+ \frac{\partial \sigma_{yy}}{\partial y} +
\frac{\partial \sigma_{yz}}{\partial z} \\ \frac{\partial
\sigma_{zx}}{\partial x} + \frac{\partial \sigma_{zy}}{\partial y} + \frac{\partial \sigma_{zz}}{\partial z} \end{array}
\right)
\end{align*}
$$

* Equations: 3

* Unknowns: 6 (components of the symmetric stress tensor)

What is missing? Material properties

## The Piola-Kirchoff stress tensors
The first Piola-Kirchoff stress tensor

$$
\boldsymbol{P} =  J \boldsymbol{\sigma} \boldsymbol{F}^{-T}
$$

gives the actual force referred to the undeformed surface area.

The second Piola-Kirchoff stress tensor

$$
\boldsymbol{S} =  J\boldsymbol{F}^{-1}\boldsymbol{\sigma} \boldsymbol{F}^{-T}
$$

is derived by mapping the force back to the undeformed geometry.

These stress tensors are only used as tools in computations. The
relevant physical quantity is the Cauchy stress $\sigma$.

## The equilibrium equation for large deformations
In terms of the Piola-Kirchoff stresses, the equilibrium equation reads

$$
\nabla \cdot \boldsymbol{P} = 0
$$

or

$$
\nabla \cdot \boldsymbol{F}\boldsymbol{S} = 0
$$

These are the versions we will use for modeling heart muscle mechanics.

## Constitutive equations

* How are the internal forces in a material related to the
  deformations?

* How are *stresses* related to *strains*?

## We have introduced several important concepts
* Strain; a precise measure of change of shape in a material
* Stress; force/area, a useful measure of forces acting on internal and external surfaces
* The fundamental equation that describes equilibrium of forces in the material

What is missing is a description of material behavior:
* Equation of motion is valid for any solid (rubber, steel, soft tissues...)
* Has six unknowns (components of the symmetric stress tensor), but only three equations (divergence of a tensor is a vector)
* We need constitutive equations to close the system

## Hooke’s generalized law
Elongation of a rod: Hooke’s experiments showed that
$$
\frac{F}{A} = E\frac{\Delta L}{L}
$$
* Stress (force) is linearly related to strain (relative displacement)

* The general form: $\sigma_{ij}=C_{ijkl}\varepsilon_{kl}$

* $C_{ijkl}$ is a fourth-order tensor (81 components)
  which describes the stiffness of the material

## Non-linear (hyper)elastic materials
For materials undergoing large elastic deformations, the stress-strain
relation is usually non-linear:
<!-- dom:FIGURE:[figs/exponential.png,width=600 frac=0.7] -->
<!-- begin figure -->

<p></p>
<img src="figs/exponential.png" width=600>

<!-- end figure -->

## The strain energy function (1)
* For all hyperelastic materials we can define a strain energy function $W$,
  normally defined in terms of the Green-Lagrange strain or its invariants

* For an elastic material following Hooke's law, the strain energy function
  can be written as
  $$
  W = \frac{1}{2} C_{ijkl}\varepsilon_{ij}\varepsilon_{kl}
  $$
  (Recall that in the small deformation case $\varepsilon_{ij} \approx E_{ij}$

## The strain energy function (2)
* The stress is obtained by taking the first derivative of the strain
  energy function with respect to the strain
  $$
  \sigma_{ij} = \frac{\partial W}{\partial \varepsilon_{ij}}
  $$

* The second derivative of the strain energy function gives the stiffness
  tensor
  $$
  C_{ijkl} = \frac{\partial^2 W}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}
  $$

## The strain energy function (3)
The same principles apply for non-linear hyperelastic materials:
* The first derivative of $W$ with respect to the Green-Lagrange strain
  gives the 2nd Piola-Kirchoff stress
  $$
  S_{ij} = \frac{\partial W}{\partial E_{ij}}
  $$

* Differentiating twice gives the tangential stiffness, often
  referred to as the *second elasticity tensor*
  $$
  C_{ijkl} = \frac{\partial^2 W}{\partial E_{ij}\partial E_{kl}}
  $$
  It is important to note that in the non-linear case this tensor is not constant,
  but depends on the deformation state. The tangential stiffness is used in
  computational techniques (Newton's method).

## The strain energy function (4)

Similar relations hold for the PK1 stress
* The first derivative of $W$ with respect to the deformation gradient
  gives the 1st Piola-Kirchoff stress
  $$
  P_{ij} = \frac{\partial W}{\partial F_{ij}}
  $$

* Differentiating twice gives an alternative stiffness tensor,
  known as the *first elasticity tensor*
  $$
  A_{ijkl} = \frac{\partial^2 W}{\partial F_{ij}\partial F_{kl}}
  $$

## Examples of strain energy functions (1)
St. Venant-Kirchoff:
$$
W(E) = \frac{\lambda}{2}(\textrm{tr}E)^2+\mu\textrm{tr}E^2
$$
Neo-Hookean:
$$
W = \frac{\mu}{2}(I_1-3)-\mu\log J+\frac{\lambda}{2}(\log J)^2
$$
where $I_1$ is the *first invariant* of the right Cauchy-Green tensor

## Examples of strain energy functions (2)
Orthotropic exponential model, Guccione et al (1995), 8 material parameters:
$$
W = \frac{1}{2}K(e^{Q}-1) + C_{compr}(J \ln J - J +1),
$$
with

$$
\begin{align*}
Q &= b_{ff}E_{ff}^2 + b_{ss}E_{ss}^2 +
  b_{nn}E_{nn}^2 + b_{fs}(E_{fs}^2+E_{sf}^2) \\
  & +b_{fn}(E_{fn}^2+E_{nf}^2)+
  b_{ns}(E_{ns}^2+E_{sn}^2),
\end{align*}
$$

## Summary; complete large deformation elasticity problem
Static problem with no body forces:

$$
\begin{align*}
-\nabla\cdot\boldsymbol{P} & = 0 \mbox{ in } \Omega \\
\boldsymbol{u} & = \boldsymbol{u}_0 \mbox{ on } \partial\Omega_D \\
\boldsymbol{P}\cdot \boldsymbol{n} &= \boldsymbol{T} \mbox{ on } \partial\Omega_T \\
\end{align*}
$$

* $\boldsymbol{u}$ is the displacement (primary unknown)

* $\boldsymbol{P} = \partial W/\partial\boldsymbol{F}$ is the first Piola–Kirchoff stress tensor, where $W$ is the strain energy

* $\boldsymbol{u}_0$ is a given boundary displacement, $\boldsymbol{T}$ is a given boundary traction (typically pressure)

* Note that $\Omega$ is the *undeformed* geometry, and differentiation ($\nabla$)
  is done with respect to the *undeformed* coordinates $\boldsymbol{X}$

## Comments on solution methods (1)
In the linear small-deformation case, we can insert the stress-strain
relation to get a compact PDE for $\boldsymbol{u}$. This is not possible for the non-linear case.
* Classical procedure

  * Derive the weak form of the equilibrium equation

  * Linearize the equation by introducing the tangent stiffness

  * Compute the tangent stiffness through a combination of manual steps
    and numerical differentiation

## Comments on solution methods (2)

* Our approach;

  * Derive the weak form

  * Employ Fenics tools for automatic linearization

## Deriving the weak form (1)
* You have seen the basic steps of the finite element method,
  introducing the weak form from a mathematical perspective

* In solid mechanics, it is common to employ a physical
  motivation;  *the principle of virtual work*

* Recall that we solve for the displacement $\boldsymbol{u}$;

    * Galerkin method $\Rightarrow$ test functions are taken from the
      same space as the solutions

    * Test functions may be interpreted as *virtual displacements*


* The result is *exactly* the same as the finite element derivations you have
  seen in earlier lectures. The only difference is the physical motivation.

## Deriving the weak form (2)
The principle of virtual work:
* We require that the total (virtual) work vanishes for all permissible choices of
  virtual displacements $\boldsymbol{v}$ (The notation $\partial\boldsymbol{u}$ is often used in the
  literature. We use $\boldsymbol{v}$ for simplicity.)

* Or in mathematical terms
  $$
  \partial W = \int_{\Omega}  (\nabla\cdot \boldsymbol{P}) \cdot \boldsymbol{v} dV
  -\int_{\partial\Omega_T} \boldsymbol{T}\cdot\boldsymbol{v}  dS= 0
  $$
  for all $\boldsymbol{v}$ in a suitable function space $V$.

* Integration by parts gives
  $$
  \int_{\Omega}(\boldsymbol{P}:\nabla \boldsymbol{v})dV - \int_{\partial
    \Omega_T}\boldsymbol{T}\cdot \boldsymbol{v} dS = 0
  $$

## Next: Apply the theory in simple test cases
* Stretching a unit cube

* A contracting cube

* Inflating a simplified left ventricle